In [222]:
import os
import re
import pandas as pd

# Limpeza dos discursos

## 1 Criando uma lista com todos os discursos

In [223]:
# Recebendo os discursos
discursos = []
for j in os.listdir('discursos'):
    file = open(f'discursos/{j}', 'r', encoding="utf-8")
    text = file.read()
    discursos.append(text)
    file.close  


In [224]:
discursos = pd.Series(discursos)

In [225]:
# #Criando lista com todos os conectivos da lingua portuguesa
# conectivos = pd.read_html('https://duvidas.dicio.com.br/conectivos/')

# a = conectivos[0]['Exemplos:']
# conectivos_lista = []
# for j in a:
#     conectivos_lista.append(j.upper())
# conectivos_lista = " ".join(conectivos_lista)

# conectivos_lista = conectivos_lista.replace('; ', ' ').replace('… ', ' ').split(' ')
# conectivos_lista = list(dict.fromkeys(conectivos_lista))
# outros_conectivos = ['OS', 'E', 'UM', 'UMA']
# for i in outros_conectivos:
#     conectivos_lista.append(i)
# conectivos_lista = sorted(conectivos_lista)

In [226]:
from stop_words import get_stop_words
conectivos_lista = get_stop_words('pt')
conectivos_lista = [sw.upper() for sw in conectivos_lista]
conectivos_lista.append('É')

## 2 Verificando se houve revisão nos discursos

In [227]:
revisado = []
for j in range(len(discursos)):
    if(re.search(r'Sem revisão', discursos[j])):
        revisado.append(False)
    else:
        revisado.append(True)
revisao = pd.Series(revisado)

In [228]:
df_discursos = {"discursos" : discursos,
                "revisao" : revisao}
df_discursos = pd.concat(df_discursos, axis=1)

## 3 Limpando os discurso

In [229]:
from stop_words import get_stop_words
conectivos_lista = []
conectivos_lista = get_stop_words('pt')
conectivos_lista.append('é')
conectivos_lista = [conectivo.upper() for conectivo in conectivos_lista]

In [230]:
#Retirando a introdução
discursos = df_discursos['discursos'].str.split('.\) -')
df_discursos['discursos'] = [i[1:] for i in discursos]

discursos_lista = []
discurso_limpo = []

for i in df_discursos['discursos']:
    discursos_lista.append(" ".join(i))

termos_discursos = []
for i in range(len(discursos_lista)):

    #Retirando os sons ambientes
    sons_ambiente = ['<tr><td align="center">(<i>Soa a campainha.</i>)</td></tr></table>', '<i>(Palmas.)</i>']
    for j in sons_ambiente:
        discursos_lista[i] = discursos_lista[i].replace(j, '')
    
    #Retirando as tags
    tags = ['<i>', '</i>', f'\n']
    for j in tags:
        discursos_lista[i] = discursos_lista[i].replace(j, '')

    #Retirando pontuacoes
    pontuacoes = ['.', ':', '...', '(', ')', '[', ']', '{', '}', '!', '?', ',', ';', '-', '_', "'", "\"", "nº", '<', '>', '\\', '|', '/', ', , ', ']\n']
    for j in pontuacoes:
        discursos_lista[i] = discursos_lista[i].replace(j, '')

    #retirando numeros
    for j in range(10):
        discursos_lista[i] = discursos_lista[i].replace(str(j), '')

    discurso_limpo.append(discursos_lista[i])
    discursos_lista[i] = discursos_lista[i].split(' ')
    
    #separando os termos e retirando conectivos
    termos = []
    for j in range(len(discursos_lista[i])):
        if (discursos_lista[i][j] != '') and (type(discursos_lista[i][j]) == str):
            termos.append(discursos_lista[i][j].upper())
    termos = [termo for termo in termos if not termo in conectivos_lista]
    termos_discursos.append(termos)


df_discursos['discursos'] = discurso_limpo
df_discursos['termos'] = [pd.Series(i) for i in termos_discursos]



In [231]:
df_discursos

,discursos,revisao,termos
0,Muito obrigado Presidente Volto mais uma vez...,False,0 OBRIGADO 1 PRESIDENTE 2 ...
1,Presidente nós da Oposição estamos defendendo...,False,0 PRESIDENTE 1 OPOSIÇÃO 2 DEF...
2,Povo brasileiro eles têm método Bolsonaro va...,False,0 POVO 1 BRASILEIRO 2 ...
3,Sr Presidente inicio dizendo à Deputada Bia K...,False,0 SR 1 PRESIDENTE 2 ...
4,Obrigado Presidente Luiz Lima Quero cumprime...,False,0 OBRIGADO 1 PRESIDENTE 2 ...
...,...,...,...
95,O PT orienta sim Presidente porque este Congr...,False,0 PT 1 ORIENTA 2 ...
96,Presidente Marcelo Ramos quero trazer aqui a ...,False,0 PRESIDENTE 1 MARCELO 2 ...
97,Sr Presidente muito obrigado pela gentileza ...,False,0 SR 1 PRESIDENTE 2 OBRI...
98,Mais uma vez eu gostaria de destacar os itens...,False,0 VEZ 1 GOSTARIA 2 DESTACA...


## 4 Contagem de termos

In [237]:
df_discursos['termos'][0].value_count()

AttributeError: 'Series' object has no attribute 'value_count'